In [64]:
# imports

import os
import io
import sys
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import subprocess


In [65]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")


if groq_api_key:
    print(f"Groq API Key exists and begins {groq_api_key[:4]}")
else:
    print("Groq API Key not set (and this is optional)")



OpenAI API Key exists and begins sk-proj-
Google API Key exists and begins AI
Groq API Key exists and begins gsk_


In [66]:
# Connect to client libraries

openai = OpenAI()

gemini_url = "https://generativelanguage.googleapis.com/v1beta/openai/"
groq_url = "https://api.groq.com/openai/v1"
ollama_url = "http://localhost:11434/v1"

gemini = OpenAI(api_key=google_api_key, base_url=gemini_url)
groq = OpenAI(api_key=groq_api_key, base_url=groq_url)
ollama = OpenAI(api_key="ollama", base_url=ollama_url)



In [ ]:
models = ["gpt-5", "gemini-2.5-pro", "qwen2.5-coder", "deepseek-coder-v2", "openai/gpt-oss-120b", ]

clients = {"gpt-5": openai, "gemini-2.5-pro": gemini, "openai/gpt-oss-120b": groq, "qwen2.5-coder": ollama, "deepseek-coder-v2": ollama,}


In [68]:
from system_info import retrieve_system_info

system_info = retrieve_system_info()
system_info

{'os': {'system': 'Darwin',
  'arch': 'arm64',
  'release': '25.1.0',
  'version': 'Darwin Kernel Version 25.1.0: Mon Oct 20 19:32:47 PDT 2025; root:xnu-12377.41.6~2/RELEASE_ARM64_T8103',
  'kernel': '25.1.0',
  'distro': None,
  'wsl': False,
  'rosetta2_translated': False,
  'target_triple': 'arm64-apple-darwin25.1.0'},
 'package_managers': ['xcode-select (CLT)', 'brew'],
 'cpu': {'brand': 'Apple M1',
  'cores_logical': 8,
  'cores_physical': 8,
  'simd': []},
 'toolchain': {'compilers': {'gcc': 'Apple clang version 17.0.0 (clang-1700.4.4.1)',
   'g++': 'Apple clang version 17.0.0 (clang-1700.4.4.1)',
   'clang': 'Apple clang version 17.0.0 (clang-1700.4.4.1)',
   'msvc_cl': ''},
  'build_tools': {'cmake': '', 'ninja': '', 'make': 'GNU Make 3.81'},
  'linkers': {'ld_lld': ''}}}

In [69]:
compile_command = ["clang++", "-std=c++17", "-Ofast", "-mcpu=native", "-flto=thin", "-fvisibility=hidden", "-DNDEBUG", "main.cpp", "-o", "main"]
run_command = ["./main"]


In [70]:
language = "C++"
system_prompt = f"""
Your task is to convert Python code into high performance {language} code.
Respond only with {language} code. Do not provide any explanation other than occasional comments.
The {language} response needs to produce an identical output in the fastest possible time.
"""

def user_prompt_for(python):
    return f"""
Port this Python code to {language} with the fastest possible implementation that produces identical output in the least time.
The system information is:
{system_info}
Your response will be written to a file called main.{language} and then compiled and executed; the compilation command is:
{compile_command}
Respond only with {language} code.
Python code to port:

```python
{python}
```
"""

In [71]:
def messages_for(python):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(python)}
    ]
 

In [72]:
def write_output(cpp):
    with open("main.cpp", "w") as f:
        f.write(cpp)

In [73]:
def port(model, python):
    try:
        client = clients[model]
        reasoning_effort = "high" if 'gpt' in model else None
        response = client.chat.completions.create(model=model, messages=messages_for(python))
        print(response)
        reply = response.choices[0].message.content
        reply = reply.replace('```cpp','').replace('```','')
        write_output(reply)
        return reply
    except Exception as e:
        print(e)

In [74]:
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(200_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [75]:
def run_python(code):
    globals_dict = {"__builtins__": __builtins__}

    buffer = io.StringIO()
    old_stdout = sys.stdout
    sys.stdout = buffer

    try:
        exec(code, globals_dict)
        output = buffer.getvalue()
    except Exception as e:
        output = f"Error: {e}"
    finally:
        sys.stdout = old_stdout

    return output

In [76]:
def compile_and_run(code):
    write_output(code)
    try:
        subprocess.run(compile_command, check=True, text=True, capture_output=True)
        run_result = subprocess.run(run_command, check=True, text=True, capture_output=True)
        return run_result.stdout
    except subprocess.CalledProcessError as e:
        return f"An error occurred:\n{e.stderr}"

In [77]:
# with gr.Blocks() as ui:
#     with gr.Row():
#         python = gr.Textbox(label="Python code:", lines=28, value=pi)
#         cpp = gr.Textbox(label="C++ code:", lines=28)
#     with gr.Row():
#         model = gr.Dropdown(models, label="Select model", value=models[0])
#         convert = gr.Button("Convert code")

#     convert.click(port, inputs=[model, python], outputs=[cpp])

# ui.launch(inbrowser=True)
# print(port("gemini-2.5-pro", pi))

In [78]:
from styles import CSS

with gr.Blocks(css=CSS, theme=gr.themes.Monochrome(), title=f"Port from Python to {language}") as ui:
    with gr.Row(equal_height=True):
        with gr.Column(scale=6):
            python = gr.Code(
                label="Python (original)",
                value=pi,
                language="python",
                lines=26
            )
        with gr.Column(scale=6):
            cpp = gr.Code(
                label=f"{language} (generated)",
                value="",
                language="cpp",
                lines=26
            )

    with gr.Row(elem_classes=["controls"]):
        python_run = gr.Button("Run Python", elem_classes=["run-btn", "py"])
        model = gr.Dropdown(models, value=models[0], show_label=False)
        convert = gr.Button(f"Port to {language}", elem_classes=["convert-btn"])
        cpp_run = gr.Button(f"Run {language}", elem_classes=["run-btn", "cpp"])

    with gr.Row(equal_height=True):
        with gr.Column(scale=6):
            python_out = gr.TextArea(label="Python result", lines=8, elem_classes=["py-out"])
        with gr.Column(scale=6):
            cpp_out = gr.TextArea(label=f"{language} result", lines=8, elem_classes=["cpp-out"])

    convert.click(fn=port, inputs=[model, python], outputs=[cpp])
    python_run.click(fn=run_python, inputs=[python], outputs=[python_out])
    cpp_run.click(fn=compile_and_run, inputs=[cpp], outputs=[cpp_out])

ui.launch(inbrowser=True)


* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.
